# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:42<00:00, 20.47s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Harbor Freight Tools Scratch and Dent Sale: Up to 45% off + free shipping\nDetails: Save up to 40% off a range of tool storage chests and stainless steel worktops. We\'ve pictured the Icon 73" Professional Top Chest for $1,259 ($540 off) Stock is limited. Items ship for free, although you\'re responsible for un-palletizing and setup. You need to call the number on the sale page to place an order for these items. Shop Now at Harbor Freight Tools\nFeatures: \nURL: https://www.dealnews.com/Harbor-Freight-Tools-Scratch-and-Dent-Sale-Up-to-45-off-free-shipping/21735179.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Adorama Warehouse Sale: Huge savings across all categories + free shipping w/ $50
Details: For a limited time, score deep savings on in stock gear. You will save on photo, video, lighting, audio, computers, and much more. Orders of $50 or more ship for free. Shop Now at Adorama
Features: 
URL: https://www.dealnews.com/Adorama-Warehouse-Sale-Huge-savings-across-all-c

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The Certified Refurbished iRobot Roomba j7+ is a smart self-emptying robot vacuum that provides a hassle-free cleaning experience. It features advanced smart mapping technology, allowing it to efficiently navigate your home and avoid obstacles. This model includes a self-cleaning cartridge filter, which helps in maintaining the suction power while significantly reducing the need for manual maintenance. Additionally, it comes with a 2-year Allstate warranty, ensuring peace of mind with your purchase.', price=180.0, url='https://www.dealnews.com/products/iRobot/iRobot-Roomba-j7-Self-Emptying-Robot-Vacuum/455442.html?iref=rss-f1912')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
print(result)

deals=[Deal(product_description='Get a certified refurbished Unlocked Apple iPhone 15 with 128GB of storage for just $474.99. This phone is available at an exceptional price and is the lowest listed price for any condition. It comes complete with a 1-year Allstate warranty, ensuring peace of mind with your purchase.', price=474.99, url='https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-15-128-GB-Smartphone/482155.html?iref=rss-c142'), Deal(product_description="This refurbished Unlocked Apple iPhone 15 Pro Max features a substantial 256GB of storage capacity. At $710, it's the best deal currently available as it has dropped significantly in price. This device also comes with a 1-year Allstate warranty, offering protection and support throughout your usage.", price=710.0, url='https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-15-Pro-Max-256-GB-Smartphone/482160.html?iref=rss-c142'), Deal(product_description='Experience high-end audio with the Samsung Galaxy Buds F